In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from glob import glob
import os
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array,array_to_img

In [2]:
from tensorflow.keras.models import load_model

autoEncoder = load_model("autoencoderV3B.h5")
autoEncoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856 

In [3]:
from keras.models import Model

# assume you have already created an autoencoder model named 'autoencoder_model'

# create a new model that takes the input of the autoencoder
encoder = Model(inputs=autoEncoder.input, outputs=autoEncoder.layers[8].output)

# print the summary of the model to verify that it has the first 5 layers
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856 

In [4]:
# Set the path to the directory containing the tif images
img_dir = '../Data/train'

# Set the path to the CSV file containing the id-label mapping
id_file = "../Data/train_labels/train_labels.csv"

# Load the id-label mapping file into a DataFrame
id_df = pd.read_csv(id_file,index_col=0)


id_df

,label
id,
f38a6374c348f90b587e046aac6079959adf3835,0
c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
755db6279dae599ebb4d39a9123cce439965282d,0
bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
068aba587a4950175d04c680d38943fd488d6a9d,0
...,...
53e9aa9d46e720bf3c6a7528d1fca3ba6e2e49f6,0
d4b854fe38b07fe2831ad73892b3cec877689576,1
3d046cead1a2a5cbe00b2b4847cfb7ba7cf5fe75,0


In [5]:
len(id_df[id_df['label'] == 1])

89117

In [6]:
len(id_df[id_df['label'] == 0])

130908

In [7]:
df = pd.concat([id_df[id_df['label'] == 1].sample(n=10000, random_state=1),id_df[id_df['label'] == 0].sample(n=10000, random_state=1)])

In [8]:
#Shuffle
df = df.sample(frac = 1)

In [9]:
sample_img = load_img(os.path.join(img_dir, os.listdir(img_dir)[0]))
img_width, img_height = sample_img.size

In [10]:
df

,label
id,
79992e5c152fdb25e9dbf2fcae7ee199fab13533,0
aa87f42e21dddcb962fe90eaf4e9ae8833b89684,1
8d68d1a0846c6826516f99ad0855f11cf17b66a5,0
9844198e52e72a1ad7f8e98814643e9d27b990d2,1
a7842ecb6c8a9fa8d6e607dd81ed245a969bb9f8,1
...,...
3a6966f3e9b6458d502ccb7cfead69874f34ea6c,1
b77e74cc45726aa0509c6178f5a527e13a8d7837,0
e1fc149e11b1961c51c57d85b46ba9af4b7f5b03,1


In [11]:
data = []
count = 0

In [12]:

# Load the first 100 RGB TIFF images and convert them to arrays
for index, row in df.iterrows():
    if(count % 1000 == 0):
        print(count)
    count = count + 1
    img = load_img(os.path.join(img_dir, index+".tif"), target_size=(img_width, img_height))
    img_array = img_to_array(img)
    data.append(img_array)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [13]:
data


data = np.array(data)


# Normalize the input data
data = data / 255.

In [21]:
p_data = encoder.predict(data)

In [14]:
# compressed_data = encoder.predict(data)

In [15]:
# compressed_data.shape

In [16]:
# compressed_data[0].shape

In [17]:
pd.Series()

C:\Users\Rohan\AppData\Local\Temp\ipykernel_22768\2031691219.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series()


Series([], dtype: float64)

In [26]:
p_data.shape.ravel()

AttributeError: 'tuple' object has no attribute 'ravel'

In [24]:
df_p = pd.DataFrame({"name": p_data})

ValueError: Per-column arrays must each be 1-dimensional